In [ ]:
import requests
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# 1. URL에서 문서 로드하기
url = "https://gist.githubusercontent.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223/raw/d72b9558a11523adbe13300b41321ecd93d331d3/document.txt"
response = requests.get(url)
response.raise_for_status()  # 요청이 성공했는지 확인
text = response.text

# 2. 문서를 Document 객체로 래핑하기
document = Document(page_content=text, metadata={"source": url})

# 3. 문서 쪼개기
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents([document])

# 4. 임베딩 생성 및 캐시
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
vectorstore = FAISS.from_documents(docs, embeddings)

# 5. 벡터 스토어 생성
retriever = vectorstore.as_retriever()

# 6. 대화 메모리 생성
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 7. 체인 연결

# LLM과 프롬프트 정의
llm = OpenAI(model_name="gpt-4o-mini-2024-07-18", temperature=0.1)
prompt = PromptTemplate(template="Use the following documents to answer the question: {documents}\nQuestion: {question}\nAnswer:", input_variables=["documents", "question"])

# LLMChain 생성
llm_chain = LLMChain(llm=llm, prompt=prompt)

# StuffDocumentsChain 생성 (document_variable_name 명시)
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="documents")

# RetrievalQA 체인 생성
qa_chain = RetrievalQA(retriever=retriever, combine_documents_chain=stuff_chain, memory=memory)

# 8. 질문하기
questions = [
    "Is Aaronson guilty?",
    "What message did he write in the table?",
    "Who is Julia?"
]

for question in questions:
    result = qa_chain.run(question)
    print(f"Q: {question}\nA: {result}\n")

# 대화 기록 확인
print("\nConversation History:")
print(memory.chat_history)